## 1- Import Libraries

In [ ]:
!pip install keras_tuner

In [5]:
import time
import numpy as np
import pandas as pd
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

## 2- Load and Inspect Dataset

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
x_train shape: (50000, 32, 32, 3) y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) y_test shape: (10000, 1)


### 1- Combine and Train/Test Split

15% test / 85% train+val

In [7]:
x_all = np.concatenate([x_train, x_test])
y_all = np.concatenate([y_train, y_test])

x_train_val, x_test, y_train_val, y_test = train_test_split(x_all, y_all, test_size=0.15, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, random_state=42)

### 2- Normalize Pixel Values

In [8]:
x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

### 3- Convert  labels to on-hot fot CategoricalCrossentropy

In [9]:
y_train_cat = to_categorical(y_train, 10)
y_val_cat = to_categorical(y_val, 10)
y_test_cat = to_categorical(y_test, 10)

## 2- Define Model

In [10]:
def build_deep_model():
    model = keras.Sequential([

        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),

        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

## 3- Train and Fit Model

In [11]:
deep_model = build_deep_model()

deep_model.fit(x_train, y_train_cat, epochs=10, validation_data=(x_val, y_val_cat), batch_size=32, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 58s 44ms/step - accuracy: 0.3281 - loss: 1.8093 - val_accuracy: 0.5160 - val_loss: 1.3259
Epoch 2/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 55s 43ms/step - accuracy: 0.5500 - loss: 1.2593 - val_accuracy: 0.6107 - val_loss: 1.0965
Epoch 3/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - accuracy: 0.6182 - loss: 1.0850 - val_accuracy: 0.6248 - val_loss: 1.0630
Epoch 4/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.6547 - loss: 0.9802 - val_accuracy: 0.6458 - val_loss: 1.0153
Epoch 5/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.6843 - loss: 0.9054 - val_accuracy: 0.6687 - val_loss: 0.9592
Epoch 6/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 85s 46ms/step - accuracy: 0.6994 - loss: 0.8517 - val_accuracy: 0.6808 - val_loss: 0.9136
Epoch 7/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 80s 45ms/step - accuracy: 0.7237 - loss: 0.7924 - val_accuracy: 0.6905 - val_loss: 0.8935
Epoch 8/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 80s 43ms/step - accuracy: 0.7400 -

## 4- Tune Hyperparameters with KerasTuner

In [12]:
def build_tuned_model(hp):
    model = keras.Sequential([

        layers.Conv2D(hp.Int('filters1', 32, 128, step=32), (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(hp.Int('filters2', 32, 128, step=32), (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(hp.Int('filters3', 32, 128, step=32), (3, 3), activation='relu'),
        layers.Flatten(),

        layers.Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [13]:
tuner = kt.RandomSearch(build_tuned_model, objective='val_accuracy', max_trials=5, executions_per_trial=1)

start_time = time.time()

tuner.search(x_train, y_train_cat, epochs=5, validation_data=(x_val, y_val_cat))

tuning_time = time.time() - start_time

best_model = tuner.get_best_models(num_models=1)[0]

print(f"Tuning Time: {tuning_time:.2f} seconds")

Trial 5 Complete [00h 11m 54s]
val_accuracy: 0.6694117784500122

Best val_accuracy So Far: 0.686274528503418
Total elapsed time: 01h 02m 27s
Tuning Time: 3746.76 seconds


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## 5- Evaluation

In [14]:
def compute_metrics(model, x, y_true):

    y_pred = model.predict(x)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)

    acc = accuracy_score(y_true_classes, y_pred_classes)
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    roc = roc_auc_score(y_true, y_pred, multi_class='ovr')

    return acc, f1, roc

### Deep Metrics

In [15]:
deep_metrics = {
    "Train": compute_metrics(deep_model, x_train, y_train_cat),
    "Validation": compute_metrics(deep_model, x_val, y_val_cat),
    "Test": compute_metrics(deep_model, x_test, y_test_cat)
}

1275/1275 ━━━━━━━━━━━━━━━━━━━━ 16s 12ms/step
319/319 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


### Tuned Metrics

In [16]:
tuned_metrics = {
    "Train": compute_metrics(best_model, x_train, y_train_cat),
    "Validation": compute_metrics(best_model, x_val, y_val_cat),
    "Test": compute_metrics(best_model, x_test, y_test_cat)
}

1275/1275 ━━━━━━━━━━━━━━━━━━━━ 37s 29ms/step
319/319 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step
282/282 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step


## 6- Wide and Deep Model

In [17]:
def build_wide_deep_model():
    inputs = layers.Input(shape=(32, 32, 3))

    # wide path
    wide = layers.Flatten()(inputs)
    wide = layers.Dense(128, activation='relu')(wide)

    # deep path
    deep = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    deep = layers.MaxPooling2D((2, 2))(deep)
    deep = layers.Conv2D(64, (3, 3), activation='relu')(deep)
    deep = layers.Flatten()(deep)
    deep = layers.Dense(64, activation='relu')(deep)

    # combine
    combined = layers.concatenate([wide, deep])
    output = layers.Dense(10, activation='softmax')(combined)

    # define and compile model
    model = keras.Model(inputs, output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

### Fit Model

In [18]:
wide_deep_model = build_wide_deep_model()

wide_deep_model.fit(x_train, y_train_cat, epochs=10, validation_data=(x_val, y_val_cat), batch_size=32, verbose=1)

Epoch 1/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 66s 50ms/step - accuracy: 0.3603 - loss: 1.7930 - val_accuracy: 0.5709 - val_loss: 1.1850
Epoch 2/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 80s 49ms/step - accuracy: 0.5964 - loss: 1.1372 - val_accuracy: 0.6146 - val_loss: 1.1126
Epoch 3/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 84s 51ms/step - accuracy: 0.6671 - loss: 0.9505 - val_accuracy: 0.6508 - val_loss: 0.9999
Epoch 4/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 81s 49ms/step - accuracy: 0.7144 - loss: 0.8162 - val_accuracy: 0.6821 - val_loss: 0.9364
Epoch 5/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 63s 49ms/step - accuracy: 0.7492 - loss: 0.7197 - val_accuracy: 0.6832 - val_loss: 0.9385
Epoch 6/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 83s 50ms/step - accuracy: 0.7777 - loss: 0.6406 - val_accuracy: 0.6760 - val_loss: 0.9809
Epoch 7/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 83s 51ms/step - accuracy: 0.8020 - loss: 0.5685 - val_accuracy: 0.6657 - val_loss: 1.0442
Epoch 8/10
1275/1275 ━━━━━━━━━━━━━━━━━━━━ 81s 50ms/step - accuracy: 0.8286 -

In [20]:
wide_deep_metrics = {
    "Train": compute_metrics(wide_deep_model, x_train, y_train_cat),
    "Validation": compute_metrics(wide_deep_model, x_val, y_val_cat),
    "Test": compute_metrics(wide_deep_model, x_test, y_test_cat)
}

1275/1275 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step
319/319 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step


## 7- Results

In [21]:
results = pd.DataFrame({
    "Dataset": ["Train", "Validation", "Test"],
    "Deep Accuracy": [deep_metrics[d][0] for d in deep_metrics],
    "Deep F1": [deep_metrics[d][1] for d in deep_metrics],
    "Deep ROC-AUC": [deep_metrics[d][2] for d in deep_metrics],

    "Tuned Accuracy": [tuned_metrics[d][0] for d in tuned_metrics],
    "Tuned F1": [tuned_metrics[d][1] for d in tuned_metrics],
    "Tuned ROC-AUC": [tuned_metrics[d][2] for d in tuned_metrics],

    "WideDeep Accuracy": [wide_deep_metrics[d][0] for d in wide_deep_metrics],
    "WideDeep F1": [wide_deep_metrics[d][1] for d in wide_deep_metrics],
    "WideDeep ROC-AUC": [wide_deep_metrics[d][2] for d in wide_deep_metrics]
})

print("\nResults Table:")
print(results)


Results Table:
      Dataset  Deep Accuracy   Deep F1  Deep ROC-AUC  Tuned Accuracy  \
0       Train       0.772328  0.770789      0.976386        0.734314   
1  Validation       0.692157  0.688466      0.951610        0.686275   
2        Test       0.677778  0.674001      0.950641        0.679000   

   Tuned F1  Tuned ROC-AUC  WideDeep Accuracy  WideDeep F1  WideDeep ROC-AUC  
0  0.733140       0.965429           0.883260     0.883756          0.993977  
1  0.684283       0.950662           0.665098     0.665201          0.943706  
2  0.676458       0.949755           0.657333     0.657678          0.943391  
